# Audio Translator 

Translating any audio into a different language, as a file.

---
## Important Installations & Setup

In [ ]:
# !pip install google-cloud-texttospeech

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = " " #add your api key

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = " " #add your credentials json file

---
## OpenAI For Transcribing


In [ ]:
from openai import OpenAI

client = OpenAI()

def transcribe_file_and_dub(audio_file_path: str):
    with open(audio_file_path, "rb") as audio_file:
        response = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )
    
    text_block = response.text
    
    print(text_block)

    text_block = translate(text_block, language="Telugu")
    
    # Change the audio name and location based on how you want it to be saved
    text_to_speech(text_block, "/Users/sharon/Desktop/telugu_audio.mp3")



---
## OpenAI For Translation

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

translation_template = """
Translate the following sentence into {language}, return ONLY the translation, nothing else.

Sentence: {sentence}
"""

output_parser = StrOutputParser()
llm = ChatOpenAI(model="gpt-5")
translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
    {"language": RunnablePassthrough(), "sentence": RunnablePassthrough()} 
    | translation_prompt
    | llm
    | output_parser
)

def translate(sentence, language):
    data_input = {"language": language, "sentence": sentence}
    translation = translation_chain.invoke(data_input)
    print(translation)
    return translation

---
## Google Cloud For Text to Speech

Languages and Voices list -> https://cloud.google.com/text-to-speech/docs/list-voices-and-types

In [41]:
from google.cloud import texttospeech

# Initialize the client once (outside the function, so it doesn't re-initialize every call)
tts_client = texttospeech.TextToSpeechClient()

def text_to_speech(text, output_file):
    
    # Prepare the input text
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Choose the voice
    voice = texttospeech.VoiceSelectionParams(
        language_code="te-IN",
        name="te-IN-Chirp3-HD-Charon"
    )

    # Configure audio output
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        effects_profile_id=["small-bluetooth-speaker-class-device"],
        speaking_rate=1.0,
    )

    # Call the API
    response = tts_client.synthesize_speech(
        input=synthesis_input,
        voice=voice,
        audio_config=audio_config
    )

    # Save to file
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
        print(f'✅ Audio content written to file "{output_file}"')




---
## Main Script


In [50]:
#audio_path = "/Users/sharon/Downloads/acts/acts17(part2).mp3"  # or .wav/.m4a, etc.

audio_file_path = "/Users/sharon/Desktop/audio.m4a"

transcribe_file_and_dub(audio_file_path)

Hi, this is Sharon.
హాయ్, నేను షారన్.
✅ Audio content written to file "/Users/sharon/Desktop/telugu_audio.mp3"
